In [34]:
import pandas as pd
import psycopg2

conn = psycopg2.connect(
  database="defaultdb",
                        host="ibd-tp1-jssantosqueiroz-badd.f.aivencloud.com",
                        user="avnadmin",
                        password="",
                        port="27074")
df = pd.read_excel('./raw_data/RELATORIO_DTB_BRASIL_MUNICIPIO.xls', skiprows=6)

# prints column names
# print(df.columns)

# base_df = df.loc[:, ['UF', 'Nome_UF', 'Código Município Completo','Município', 'Nome_Município']]

query = """
DROP TABLE IF EXISTS municipios;
CREATE TABLE municipios (
    uf int REFERENCES estados(cod_uf),
    codigo INT PRIMARY KEY UNIQUE NOT NULL,
    nome_municipio VARCHAR(255),
    codigo_completo INT
);
"""


for i in range(df.shape[0]):
    row = df.iloc[i]
    uf = row['UF']
    codigo = row['Código Município Completo']
    nome_municipio = row['Município']
    codigo_completo = row['Código Município Completo']
    query += f"INSERT INTO municipios VALUES ('{uf}', {codigo}, '{nome_municipio}', {codigo_completo});\n"
    conn.cursor().execute(query)
    conn.commit()


conn.close()


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
